In [1]:
#importing necessary libraries
from transformers import AutoProcessor, FlavaModel, FlavaImageModel, AutoImageProcessor, FlavaFeatureExtractor
import numpy as np
from torchmultimodal.models.flava.model import flava_model_for_classification
from transformers import FlavaProcessor, FlavaForPreTraining
import torchvision
import torch
import torch.nn as nn




/home/IAIS/jraghu/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Checking for GPU and settingt he device as GPU if avaialble
if torch.cuda.is_available():
    print("CUDA is available on this system.")
    device = torch.device("cuda")
else:
    print("CUDA is not available on this system.")
    device = torch.device("cpu")




CUDA is available on this system.


In [3]:
print(torch.cuda.memory_summary(device=device, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

In [4]:
from tqdm import tqdm
from avalanche.benchmarks.generators import nc_benchmark, ni_benchmark
from avalanche.benchmarks.datasets import MNIST, CUB200

from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop
import torchvision.transforms as transforms
from torch.utils.data import random_split


resize_transform = transforms.Resize((224, 224))
grayscale_transform = transforms.Grayscale(num_output_channels=3)
train_transform = Compose([
    resize_transform,
    grayscale_transform,
    ToTensor()
    
])

test_transform = Compose([
    resize_transform,
    grayscale_transform,
    ToTensor()

])


mnist_train = MNIST(
    './data/mnist', train=True, download=True, transform=train_transform
)

mnist_test = MNIST(
    './data/mnist', train=False, download=True, transform=test_transform
)

# cub_train = CUB200(
#     './data/cub200', train=True, download=True, transform=train_transform
# )

# cub_test = CUB200(
#     './data/cub200', train=False, download=True,  transform=test_transform
# )


In [5]:
#Checking for image shape and size
# Get the first image from the dataset
image, label = mnist_train[0]

# Check the size and shape of the image
size = image.size
shape = image.shape

print(f"Image Size: {size}")
print(f"Image Shape: {shape}")



Image Size: <built-in method size of Tensor object at 0x7f6622b30310>
Image Shape: torch.Size([3, 224, 224])


In [13]:
#Classsic class incremental creation from MNIST dataset
scenario_CIL = nc_benchmark(
    mnist_train, mnist_test, n_experiences=5, shuffle=True, seed=1234,
    task_labels=True
)


#Classsic Domain incremental creation from MNIST dataset
scenario_DIL = ni_benchmark(
    mnist_train, mnist_test, n_experiences=3, shuffle=False, seed=42,
    balance_experiences=True 
)


In [8]:
#creating a FLAVA custom class which calls the pretrained image model (IM encoder only) and returns the features/embeddings instead of a flava basepooling type

class flava_custom(torch.nn.Module):
    def __init__(self, num_classes, device='cuda'):
        super().__init__()
        #self.image_processor = AutoImageProcessor.from_pretrained("facebook/flava-full")
        #self.model_custom = FlavaImageModel.from_pretrained("facebook/flava-full").to("cuda")

        self.image_processor = FlavaFeatureExtractor.from_pretrained("facebook/flava-full") 
        self.model_custom = FlavaModel.from_pretrained("facebook/flava-full", low_cpu_mem_usage=True).eval().cuda()

        #self.image_processor = FlavaProcessor.from_pretrained("facebook/flava-full", return_codebook_pixels=True)
        #self.model_custom = FlavaForPreTraining.from_pretrained("facebook/flava-full").to("cuda")
        
        
        #self.model_custom = FlavaModel.from_pretrained("facebook/flava-full").eval().to("cuda")
        #self.image_processor = AutoProcessor.from_pretrained("facebook/flava-full")
        self.num_classes = num_classes
        self.device = device

    def forward(self, x):
        if not isinstance(x, list):
            x = list(x.cuda())
         
        
        inputs = self.image_processor(images=x, return_tensors="pt").to("cuda")
        #with torch.no_grad():
        outputs = self.model_custom.get_image_features(**inputs)[:, 0, :] 
        
        return outputs  #.last_hidden_state[:, 0, :]
       



        # score = outputs.last_hidden_state[:, 0, :] 

        # # Passing through a fully connected layer and the number of output classes
        # fc_layer = nn.Linear(in_features=score.size(1), out_features=self.num_classes).to(self.device)

        # # # Get the logits by passing image_features through the fc_layer
        # logits = fc_layer(score).to(self.device)
        
        
        # return logits  #last_hidden_state[:, 0, :]


In [9]:
import torch
image_tensor_with_batch = torch.unsqueeze(mnist_train[0][0], 0)
custom_DIL = flava_custom(num_classes=scenario_CIL.n_classes)
f = custom_DIL.forward(x=image_tensor_with_batch)
print(f.size)
input_ids = torch.argmax(f, dim=1).flatten(0) #important line to convert labels to match dimensions
loss = torch.nn.functional.cross_entropy(f, input_ids)
loss


/home/IAIS/jraghu/.local/lib/python3.9/site-packages/transformers/models/flava/feature_extraction_flava.py:28: FutureWarning: The class FlavaFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use FlavaImageProcessor instead.
  warnings.warn(
`text_config_dict` is provided which will be used to initialize `FlavaTextConfig`. The value `text_config["id2label"]` will be overriden.
`multimodal_config_dict` is provided which will be used to initialize `FlavaMultimodalConfig`. The value `multimodal_config["id2label"]` will be overriden.
`image_codebook_config_dict` is provided which will be used to initialize `FlavaImageCodebookConfig`. The value `image_codebook_config["id2label"]` will be overriden.


<built-in method size of Tensor object at 0x7f65d0fac630>


tensor(6.3674, device='cuda:0', grad_fn=<NllLossBackward0>)

Domain incremental setting on SplitMNIST using EWC

In [10]:
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics, amca_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin, EWCPlugin
from avalanche.training.supervised import Naive, EWC
from transformers import AutoImageProcessor, ViTModel
from avalanche.training.templates import SupervisedTemplate


# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.


custom_DIL = flava_custom(num_classes=10)

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    amca_metrics(),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario_DIL.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger]#, tb_logger]
)


# CREATE THE STRATEGY INSTANCE (EWC)
cl_strategy_DIL = EWC(
    custom_DIL, Adam(custom_DIL.parameters(), lr=0.1),
    CrossEntropyLoss(), ewc_lambda=0.4, train_mb_size=100, train_epochs=3, eval_mb_size=25,
    evaluator=eval_plugin, device=device) 

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario_DIL.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy_DIL.train(experience)
    print('Training completed')

print('Computing accuracy on the whole test set')
# test also returns a dictionary which contains all the metric values
results.append(cl_strategy_DIL.eval(scenario_DIL.test_stream))

`text_config_dict` is provided which will be used to initialize `FlavaTextConfig`. The value `text_config["id2label"]` will be overriden.
`multimodal_config_dict` is provided which will be used to initialize `FlavaMultimodalConfig`. The value `multimodal_config["id2label"]` will be overriden.
`image_codebook_config_dict` is provided which will be used to initialize `FlavaImageCodebookConfig`. The value `image_codebook_config["id2label"]` will be overriden.


Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-- >> Start of training phase << --
100%|██████████| 201/201 [07:27<00:00,  2.22s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2447198.0889
	DiskUsage_MB/train_phase/train_stream/Task000 = 2447198.0889
	Loss_Epoch/train_phase/train_stream/Task000 = 5.1775
	Loss_MB/train_phase/train_stream/Task000 = 2.1713
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0108
	Time_Epoch/train_phase/train_stream/Task000 = 446.9590
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0984
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.2500
100%|██████████| 201/201 [07:27<00:00,  2.23s/it]
Epoch 1 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2447198.0889
	DiskUsage_MB/train_phase/train_stream/Task000 = 2447198.0889
	Loss_Epoch/train_phase/train_stream/Task000 = 2.3835
	Loss_MB/train_phase/train_stream/Task000 = 2.1345
	RunningTime_Epoch/train_phase/train_stream/

TIL setting with EWC

In [11]:
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics, amca_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin, EWCPlugin
from avalanche.training.supervised import Naive, EWC
from transformers import AutoImageProcessor, ViTModel
from avalanche.training.templates import SupervisedTemplate


# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.


custom_TIL = flava_custom(num_classes=10)

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    amca_metrics(),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario_CIL.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger]#, tb_logger]
)


# CREATE THE STRATEGY INSTANCE (EWC)
cl_strategy_TIL = EWC(
    custom_TIL, SGD(custom_TIL.parameters(), lr=0.01, momentum=0.9),
    CrossEntropyLoss(), ewc_lambda=0.4, train_mb_size=100, train_epochs=3, eval_mb_size=25,
    evaluator=eval_plugin, device=device) 

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario_CIL.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy_TIL.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy_TIL.eval(scenario_CIL.test_stream))

`text_config_dict` is provided which will be used to initialize `FlavaTextConfig`. The value `text_config["id2label"]` will be overriden.
`multimodal_config_dict` is provided which will be used to initialize `FlavaMultimodalConfig`. The value `multimodal_config["id2label"]` will be overriden.
`image_codebook_config_dict` is provided which will be used to initialize `FlavaImageCodebookConfig`. The value `image_codebook_config["id2label"]` will be overriden.


Starting experiment...
Start of experience:  0
Current Classes:  [4, 5]
-- >> Start of training phase << --
100%|██████████| 113/113 [04:11<00:00,  2.23s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2730897.1426
	DiskUsage_MB/train_phase/train_stream/Task000 = 2730897.1426
	Loss_Epoch/train_phase/train_stream/Task000 = 1.2708
	Loss_MB/train_phase/train_stream/Task000 = 0.7144
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0181
	Time_Epoch/train_phase/train_stream/Task000 = 251.3411
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4903
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.5238
100%|██████████| 113/113 [04:14<00:00,  2.25s/it]
Epoch 1 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2730897.1426
	DiskUsage_MB/train_phase/train_stream/Task000 = 2730897.1426
	Loss_Epoch/train_phase/train_stream/Task000 = 0.7066
	Loss_MB/train_phase/train_stream/Task000 = 0.6952
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0170
	Time_E

EWC DIL on SMNIST using a MLP on top of custom_flava

In [10]:
import torch
import torch.nn as nn

class MLPClassificationHead(flava_custom):
    def __init__(self, num_classes, hidden_dim, device='cuda'):
        super().__init__(num_classes=num_classes, device=device)
        
        # Add an MLP classification head layers
        self.classification_head = nn.Sequential(
            nn.Linear(self.model_custom.config.hidden_size, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        # Call the forward method of the base class
        features = super().forward(x)
        
        # Pass the features through the classification head
        logits = self.classification_head(features)
        
        return logits


In [11]:
num_classes = 10
hidden_dim = 256

#model = MLPClassificationHead(num_classes=num_classes, hidden_dim=hidden_dim)


In [12]:
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics, amca_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin, EWCPlugin
from avalanche.training.supervised import Naive, EWC
from transformers import AutoImageProcessor, ViTModel
from avalanche.training.templates import SupervisedTemplate


# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.


custom_DIL_MLP =  MLPClassificationHead(num_classes=num_classes, hidden_dim=hidden_dim) #flava_custom(num_classes=10)

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    amca_metrics(),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario_DIL.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger]#, tb_logger]
)


# CREATE THE STRATEGY INSTANCE (EWC)
cl_strategy_DIL = EWC(
    custom_DIL_MLP, SGD(custom_DIL_MLP.parameters(), lr=0.01),
    CrossEntropyLoss(), ewc_lambda=0.4, train_mb_size=100, train_epochs=3, eval_mb_size=25,
    evaluator=eval_plugin, device=device) 

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario_DIL.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy_DIL.train(experience)
    print('Training completed')

print('Computing accuracy on the whole test set')
# test also returns a dictionary which contains all the metric values
results.append(cl_strategy_DIL.eval(scenario_DIL.test_stream))

/home/IAIS/jraghu/.local/lib/python3.9/site-packages/transformers/models/flava/feature_extraction_flava.py:28: FutureWarning: The class FlavaFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use FlavaImageProcessor instead.
  warnings.warn(
`text_config_dict` is provided which will be used to initialize `FlavaTextConfig`. The value `text_config["id2label"]` will be overriden.
`multimodal_config_dict` is provided which will be used to initialize `FlavaMultimodalConfig`. The value `multimodal_config["id2label"]` will be overriden.
`image_codebook_config_dict` is provided which will be used to initialize `FlavaImageCodebookConfig`. The value `image_codebook_config["id2label"]` will be overriden.


Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-- >> Start of training phase << --
100%|██████████| 201/201 [07:30<00:00,  2.24s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2730915.9590
	DiskUsage_MB/train_phase/train_stream/Task000 = 2730915.9590
	Loss_Epoch/train_phase/train_stream/Task000 = 2.2887
	Loss_MB/train_phase/train_stream/Task000 = 2.2880
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0103
	Time_Epoch/train_phase/train_stream/Task000 = 449.8511
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.1701
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.0000
100%|██████████| 201/201 [07:30<00:00,  2.24s/it]
Epoch 1 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2730915.9590
	DiskUsage_MB/train_phase/train_stream/Task000 = 2730915.9590
	Loss_Epoch/train_phase/train_stream/Task000 = 2.2729
	Loss_MB/train_phase/train_stream/Task000 = 2.2789
	RunningTime_Epoch/train_phase/train_stream/

EWC on ViT model

In [ ]:
import timm
model_vit = timm.create_model('vit_base_patch16_224_in21k', num_classes=10, pretrained=True)

/home/IAIS/jraghu/.local/lib/python3.9/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name vit_base_patch16_224_in21k to current vit_base_patch16_224.augreg_in21k.
  model = create_fn(


In [8]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics, amca_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.supervised import Naive, EWC
from transformers import AutoImageProcessor, ViTModel


# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.



# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    amca_metrics(),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario_DIL.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger]#, tb_logger]
)

# CREATE THE STRATEGY INSTANCE (EWC)
cl_strategy_DIL = EWC(
    model_vit, SGD(model_vit.parameters(), lr=0.01, momentum=0.9),
    CrossEntropyLoss(), ewc_lambda=0.4, train_mb_size=100, train_epochs=3, eval_mb_size=25,
    evaluator=eval_plugin, device=device)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario_DIL.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy_DIL.train(experience)
    print('Training completed')

print('Computing accuracy on the whole test set')
# test also returns a dictionary which contains all the metric values
results.append(cl_strategy_DIL.eval(scenario_DIL.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-- >> Start of training phase << --
100%|██████████| 121/121 [02:49<00:00,  1.40s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2447257.3984
	DiskUsage_MB/train_phase/train_stream/Task000 = 2447257.3984
	Loss_Epoch/train_phase/train_stream/Task000 = 9.3004
	Loss_MB/train_phase/train_stream/Task000 = 2.2735
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0187
	Time_Epoch/train_phase/train_stream/Task000 = 169.4870
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.1025
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.2000
100%|██████████| 121/121 [03:03<00:00,  1.52s/it]
Epoch 1 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2447257.3984
	DiskUsage_MB/train_phase/train_stream/Task000 = 2447257.3984
	Loss_Epoch/train_phase/train_stream/Task000 = 2.4527
	Loss_MB/train_phase/train_stream/Task000 = 2.4441
	RunningTime_Epoch/train_phase/train_stream/

EWC on CLIP

In [7]:
from PIL import Image
import requests
from transformers import AutoProcessor, CLIPVisionModel


In [8]:

class clip_custom(torch.nn.Module):
    def __init__(self, num_classes, device='cuda'):
        super().__init__()
        #self.image_processor = AutoImageProcessor.from_pretrained("facebook/flava-full")
        #self.model_custom = FlavaImageModel.from_pretrained("facebook/flava-full").to("cuda")

        self.image_processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
        self.model_custom = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32").to("cuda")

        self.num_classes = num_classes
        self.device = device

    def forward(self, x):
        if not isinstance(x, list):
            x = list(x.cuda())
         
        
        inputs = self.image_processor(images=x, return_tensors="pt").to("cuda")
        #with torch.no_grad():
        outputs = self.model_custom(**inputs)
        
        return outputs.last_hidden_state[:, 0, :]

In [10]:
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics, amca_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin, EWCPlugin
from avalanche.training.supervised import Naive, EWC
from transformers import AutoImageProcessor, ViTModel
from avalanche.training.templates import SupervisedTemplate


# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.


custom_DIL_clip = clip_custom(num_classes=10)

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    amca_metrics(),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario_DIL.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger]#, tb_logger]
)


# CREATE THE STRATEGY INSTANCE (EWC)
cl_strategy_DIL = EWC(
    custom_DIL_clip, SGD(custom_DIL_clip.parameters(), lr=0.1),
    CrossEntropyLoss(), ewc_lambda=0.4, train_mb_size=100, train_epochs=3, eval_mb_size=25,
    evaluator=eval_plugin, device=device) 

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario_DIL.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy_DIL.train(experience)
    print('Training completed')

print('Computing accuracy on the whole test set')
# test also returns a dictionary which contains all the metric values
results.append(cl_strategy_DIL.eval(scenario_DIL.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-- >> Start of training phase << --
100%|██████████| 201/201 [05:08<00:00,  1.53s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2729632.3066
	DiskUsage_MB/train_phase/train_stream/Task000 = 2729632.3066
	Loss_Epoch/train_phase/train_stream/Task000 = nan
	Loss_MB/train_phase/train_stream/Task000 = nan
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0063
	Time_Epoch/train_phase/train_stream/Task000 = 308.1033
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0953
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.0000
100%|██████████| 201/201 [05:09<00:00,  1.54s/it]
Epoch 1 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2729632.3066
	DiskUsage_MB/train_phase/train_stream/Task000 = 2729632.3066
	Loss_Epoch/train_phase/train_stream/Task000 = nan
	Loss_MB/train_phase/train_stream/Task000 = nan
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.

CL with Vision transformer but in the same custom class format

In [7]:
from transformers import AutoImageProcessor, ViTForImageClassification, ViTModel

import torch

from datasets import load_dataset

In [8]:
class vit_custom_logit(torch.nn.Module):
    def __init__(self, num_classes, device='cuda'):
        super().__init__()
        #self.image_processor = AutoImageProcessor.from_pretrained("facebook/flava-full")
        #self.model_custom = FlavaImageModel.from_pretrained("facebook/flava-full").to("cuda")

        self.image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
        self.model_custom = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224").to("cuda")

        self.num_classes = num_classes
        self.device = device

    def forward(self, x):
        if not isinstance(x, list):
            x = list(x.cuda())
         
        
        inputs = self.image_processor(images=x, return_tensors="pt").to("cuda")
        #with torch.no_grad():
        outputs = self.model_custom(**inputs).logits
        
        return outputs

In [9]:
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics, amca_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin, EWCPlugin
from avalanche.training.supervised import Naive, EWC
from transformers import AutoImageProcessor, ViTModel
from avalanche.training.templates import SupervisedTemplate


# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.


custom_DIL_vit = vit_custom_logit(num_classes=10)

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    amca_metrics(),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario_DIL.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger]#, tb_logger]
)


# CREATE THE STRATEGY INSTANCE (EWC)
cl_strategy_DIL = EWC(
    custom_DIL_vit, SGD(custom_DIL_vit.parameters(), lr=0.01),
    CrossEntropyLoss(), ewc_lambda=0.4, train_mb_size=100, train_epochs=3, eval_mb_size=25,
    evaluator=eval_plugin, device=device) 

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario_DIL.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy_DIL.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy_DIL.eval(scenario_DIL.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
-- >> Start of training phase << --
100%|██████████| 201/201 [07:16<00:00,  2.17s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2730870.9688
	DiskUsage_MB/train_phase/train_stream/Task000 = 2730870.9688
	Loss_Epoch/train_phase/train_stream/Task000 = 2.6704
	Loss_MB/train_phase/train_stream/Task000 = 2.1703
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0160
	Time_Epoch/train_phase/train_stream/Task000 = 436.4763
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.1006
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.5000
100%|██████████| 201/201 [07:10<00:00,  2.14s/it]
Epoch 1 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 2730870.9688
	DiskUsage_MB/train_phase/train_stream/Task000 = 2730870.9688
	Loss_Epoch/train_phase/train_stream/Task000 = 2.3615
	Loss_MB/train_phase/train_stream/Task000 = 2.4707
	RunningTime_Epoch/train_phase/train_stream/

KeyboardInterrupt: 

Custom train functions

In [5]:
from continualai.colab.scripts import mnist
mnist.init()
from tqdm import tqdm

Files already downloaded!


In [6]:


x_train, t_train, x_test, t_test = mnist.load()

print("x_train dim and type: ", x_train.shape, x_train.dtype)
print("t_train dim and type: ", t_train.shape, t_train.dtype)
print("x_test dim and type: ", x_test.shape, x_test.dtype)
print("t_test dim and type: ", t_test.shape, t_test.dtype)



x_train dim and type:  (60000, 1, 28, 28) float32
t_train dim and type:  (60000,) uint8
x_test dim and type:  (10000, 1, 28, 28) float32
t_test dim and type:  (10000,) uint8


In [7]:

import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [8]:
def train(model, device, dataset, optimizer, epoch):
    model.train()
    
    for img, label in tqdm(dataset):
      
      #print(i)
      image_tensor_with_batch = torch.unsqueeze(img, 0)
      optimizer.zero_grad()

      output = model.forward(image_tensor_with_batch)

      input_ids = torch.argmax(output, dim=1).flatten(0)
      loss = F.cross_entropy(output, input_ids)
      loss.backward()
      optimizer.step()
      #print(loss.item())
    print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, loss.item()))

def test(model, device, dataset):
    model.eval()
    test_loss = 0
    correct = 0
    for img, label in (dataset):
      
      with torch.no_grad():
        #x, y = torch.from_numpy(x_test[start:end]), torch.from_numpy(t_test[start:end]).long()
        #x, y = x.to(device), y.to(device)
        image_tensor_with_batch = torch.unsqueeze(img, 0)
        output = model.forward(image_tensor_with_batch)
        
        # 'label' is an integer
        label_array = np.array([label])
        y = torch.from_numpy(label_array).to("cuda")

        test_loss += F.cross_entropy(output, y).item() # sum up batch loss
        pred = output.max(1, keepdim=True)[1] # get the index of the max logit
        correct += pred.eq(y.view_as(pred)).sum().item()

    test_loss /= len(dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(dataset),
        100. * correct / len(dataset)))
    return 100. * correct / len(dataset)

In [8]:
x_train_rgb = np.concatenate([x_train, x_train, x_train], axis=1)

In [9]:
x_test_rgb  = np.concatenate([x_test, x_test, x_test], axis=1)

In [10]:
def train_new(model, device, x_train, t_train, optimizer, epoch):
    model.train()
    
    for start in range(0, len(t_train)-1, 100):
      end = start + 100
      x, y = torch.from_numpy(x_train[start:end]), torch.from_numpy(t_train[start:end]).long()
      x, y = x.to(device), y.to(device)
      
      optimizer.zero_grad()

      output = model(x)
      loss = F.cross_entropy(output, y)
      loss.backward()
      optimizer.step()
      #print(loss.item())
    print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, loss.item()))

def test_new(model, device, x_test, t_test):
    model.eval()
    test_loss = 0
    correct = 0
    for start in range(0, len(t_test)-1, 256):
      end = start + 256
      with torch.no_grad():
        x, y = torch.from_numpy(x_test[start:end]), torch.from_numpy(t_test[start:end]).long()
        x, y = x.to(device), y.to(device)
        output = model(x)
        test_loss += F.cross_entropy(output, y).item() # sum up batch loss
        pred = output.max(1, keepdim=True)[1] # get the index of the max logit
        correct += pred.eq(y.view_as(pred)).sum().item()

    test_loss /= len(t_test)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(t_test),
        100. * correct / len(t_test)))
    return 100. * correct / len(t_test)

In [11]:
trial_flava = flava_custom(num_classes=10)

/home/IAIS/jraghu/.local/lib/python3.9/site-packages/transformers/models/flava/feature_extraction_flava.py:28: FutureWarning: The class FlavaFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use FlavaImageProcessor instead.
  warnings.warn(
`text_config_dict` is provided which will be used to initialize `FlavaTextConfig`. The value `text_config["id2label"]` will be overriden.
`multimodal_config_dict` is provided which will be used to initialize `FlavaMultimodalConfig`. The value `multimodal_config["id2label"]` will be overriden.
`image_codebook_config_dict` is provided which will be used to initialize `FlavaImageCodebookConfig`. The value `image_codebook_config["id2label"]` will be overriden.


In [12]:

optimizer = optim.SGD(trial_flava.parameters(), lr=0.01, momentum=0.9)

In [13]:
for epoch in range(1, 3):
  train_new(trial_flava, device, x_train_rgb, t_train, optimizer, epoch)
#test_new(trial_flava, device, x_test, t_test)

Train Epoch: 1 	Loss: 2.312119
Train Epoch: 2 	Loss: 2.310108


In [15]:
test_new(trial_flava, device, x_test_rgb, t_test)

Test set: Average loss: 0.0092, Accuracy: 958/10000 (10%)



9.58

In [13]:
# import torch
# from avalanche.benchmarks.datasets import MNIST
# from torchvision.transforms import Compose, ToTensor, Lambda, RandomOrder

# resize_transform = transforms.Resize((224, 224))
# grayscale_transform = transforms.Grayscale(num_output_channels=3)
# RandomHorizontalFlip_transform = transforms.RandomHorizontalFlip(p=0.5)
# RandomVerticalFlip_transform = transforms.RandomVerticalFlip(p=0.5)
# RandomRotation_transform = transforms.RandomRotation(degrees=90)

# randomtransform = transforms.RandomOrder([RandomHorizontalFlip_transform, RandomVerticalFlip_transform, RandomRotation_transform])
# train_transform = Compose([
#     resize_transform,
#     grayscale_transform,
#     randomtransform,
#     ToTensor()
    
# ])

# # Apply the custom transform to the dataset
# permuted_mnist_train = MNIST(root='./data/mnist', train=True, download=True,  transform=train_transform)
# permuted_mnist_test = MNIST(root='./data/mnist', train=False, download=True,  transform=train_transform)
def permute_mnist(mnist, seed):
    """ Given the training set, permute pixels of each img the same way. """

    np.random.seed(seed)
    print("starting permutation...")
    h = w = 28
    perm_inds = list(range(h*w))
    np.random.shuffle(perm_inds)
    # print(perm_inds)
    perm_mnist = []
    for set in mnist:
        num_img = set.shape[0]
        flat_set = set.reshape(num_img, w * h)
        perm_mnist.append(flat_set[:, perm_inds].reshape(num_img, 1, w, h))
    print("done.")
    return perm_mnist

In [14]:
x_train2, x_test2 = permute_mnist([x_train, x_test], 0)
x_test2_rgb  = np.concatenate([x_test2, x_test2, x_test2], axis=1)
x_train2_rgb = np.concatenate([x_train2, x_train2, x_train2], axis=1)

starting permutation...
done.


In [22]:
print("Testing on the second task:")
test_new(trial_flava, device, x_test2_rgb, t_test)

Testing on the second task:
Test set: Average loss: 0.0092, Accuracy: 958/10000 (10%)



9.58

In [15]:
# task 1
task_1 = [(x_train_rgb, t_train), (x_test_rgb, t_test)]

# task 2
task_2 = [(x_train2_rgb, t_train), (x_test2_rgb, t_test)]

fisher_dict = {}
optpar_dict = {}
ewc_lambda = 0.4


# task list
tasks = [task_1, task_2]

In [16]:
def on_task_update(task_id, x_mem, t_mem):

  model.train()
  optimizer.zero_grad()
  
  # accumulating gradients
  for start in range(0, len(t_mem)-1, 100):
      end = start + 100
      x, y = torch.from_numpy(x_mem[start:end]), torch.from_numpy(t_mem[start:end]).long()
      x, y = x.to(device), y.to(device)
      output = model(x)
      loss = F.cross_entropy(output, y)
      loss.backward()

  fisher_dict[task_id] = {}
  optpar_dict[task_id] = {}

  # gradients accumulated can be used to calculate fisher
  for name, param in model.named_parameters():
    
    optpar_dict[task_id][name] = param.data.clone()
    fisher_dict[task_id][name] = param.grad.data.clone().pow(2)

In [17]:
def train_ewc(model, device, task_id, x_train, t_train, optimizer, epoch):
    model.train()

    for start in range(0, len(t_train)-1, 100):
      end = start + 100
      x, y = torch.from_numpy(x_train[start:end]), torch.from_numpy(t_train[start:end]).long()
      x, y = x.to(device), y.to(device)
      
      optimizer.zero_grad()

      output = model(x)
      loss = F.cross_entropy(output, y)
      
      ### magic here! :-)
      for task in range(task_id):
        for name, param in model.named_parameters():
          fisher = fisher_dict[task][name]
          optpar = optpar_dict[task][name]
          loss += (fisher * (optpar - param).pow(2)).sum() * ewc_lambda
      
      loss.backward()
      optimizer.step()
      #print(loss.item())
    print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, loss.item()))

In [18]:


ewc_accs = []
for id, task in enumerate(tasks):
  avg_acc = 0
  print("Training on task: ", id)
  
  (x_train_rgb, t_train), _ = task
  
  for epoch in range(1, 3):
    train_ewc(trial_flava, device, id, x_train_rgb, t_train, optimizer, epoch)
  on_task_update(id, x_train_rgb, t_train)
    
  for id_test, task in enumerate(tasks):
    print("Testing on task: ", id_test)
    _, (x_test_rgb, t_test) = task
    acc = test_new(trial_flava, device, x_test_rgb, t_test)
    avg_acc = avg_acc + acc
   
  print("Avg acc: ", avg_acc / 3)
  ewc_accs.append(avg_acc / 3)



Training on task:  0
Train Epoch: 1 	Loss: 2.312119
Train Epoch: 2 	Loss: 2.310108


NameError: name 'model' is not defined